In [ ]:
from config.model_config import DATASET_CONFIG, PREPROCESS_CONFIG
from application.controllers.preprocess_tools import STFT, Scaler
import application.controllers.utility as sp
from preprocess.utility import read, to_mono
import numpy as np
import os
import torch
import norbert
from train.model import LSTM_Model

In [ ]:
# transformation object
transform = STFT(sr=DATASET_CONFIG.SR,
                 n_per_seg=DATASET_CONFIG.N_PER_SEG,
                 n_overlap=DATASET_CONFIG.N_OVERLAP)

# Scaler object
scaler = Scaler()

In [ ]:
# time series data of mixture
data, sr = read(track_dir, stereo=True)
print("Mixture file time series data shape: ", data.shape)

data_mix = to_mono(data)
print("Shape of mono mixture time series data: ", data_mix.shape)

In [ ]:
x_tf = transform.stft(data_mix.T)
# get spectrogram of STFT i.e., |Xi|
x_mix_stft = np.abs(x_tf)
print(x_mix_stft.shape)

In [ ]:
# scaling the values to 0 to 1
X_scaled = scaler.scale(x_mix_stft)
print("Scaled data shape :", X_scaled.shape)
print("Scaled data min :", np.max(X_scaled))
print("Scaled data max :", np.max(X_scaled))
print("Scaled data mean :", np.mean(X_scaled))